<a href="https://colab.research.google.com/github/AlisterVento/PI-01.MLOps/blob/main/user_reviews_Analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ETL del archivo User_reviews_Analisis**

## **1.Cargando Las Librerías Necesarias**

In [ ]:
!pip install emoji --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import json as json  #Cargaremos json para manejar los datos de tipo json",
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import ast #Cargaremos ast para poder extraer datos de json",
import datetime #Cargaremos datetime para poder extraer la fecha de las columnas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive",
import re , emoji #Cargaremos re y emoji para poder usar expresiones regulares",
import nltk #Cargaremos nltk para poder usar expresiones regulares",
#Recursos necesarios para el procesamiento de texto
nltk.download('stopwords')
nltk.download('punkt')

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


## **2.Fase ETL**

### 2.1.Carga de Datos

In [ ]:
url02=r'/content/drive/MyDrive/2023/Henry/MLOps Henry/Data/user_reviews.json.gz'
filas=[]
with gzip.open(url02) as file:
    for line in file.readlines():
        line = line.decode('MacRoman')
        filas.append(ast.literal_eval(line))

df_user_reviews = pd.DataFrame(filas)
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### 2.2.Análisis De Descriptivo De Los Datos

#### 2.2.1.Funciones Para El Analisis Descriptivo De Los Datos

In [ ]:
def check_df(dataset, head = 5):
    """
    Funcion creada para tener una vista general de la base de datos
    """
    print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
    print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

    print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
    print(dataset.info(), end = '\n'*2)# Informacion general

    print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
    print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

    print('*'*30 + 'Mostrar las primeras 5 filas' + '*'*30, end = '\n'*2)
    dataset.head()# Mostrar las primeras 5 filas



#### 2.2.2.Descripcion De Los Datos

In [ ]:
check_df(df_user_reviews)

******************************Forma de la base de datos******************************

(25799, 3)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB
None

******************************NaN cantidad de nulos******************************

user_id     0
user_url    0
reviews     0
dtype: int64

******************************Mostrar las primeras 5 filas******************************



Procederemos a hacer un explode ala columna reviews

In [ ]:
df_user_reviews =  df_user_reviews.explode('reviews')
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


In [ ]:
df_user_reviews.dropna(subset =  ['reviews'],inplace=True)

In [ ]:
for elemento,diccionario in df_user_reviews['reviews'].head().items():
    df=pd.DataFrame(columns = diccionario.keys())
df

,funny,posted,last_edited,item_id,helpful,recommend,review


In [ ]:
new_rows = []

# Iterate over the dictionaries in the 'reviews' column
for elemento, diccionario in df_user_reviews['reviews'].items():
    new_row = pd.DataFrame(diccionario, index=[elemento])  # Create a new row DataFrame
    new_rows.append(new_row)  # Append the new row to the list

# Concatenate the list of new rows with the original DataFrame
result_df = pd.concat([df, *new_rows])

# Reset index
df_user_reviews=df_user_reviews.join(result_df)


df_user_reviews.drop_duplicates(subset=['user_id','posted'],inplace=True)


In [ ]:
df_user_reviews.drop(columns=['reviews'],inplace=True)

Vista final despues de desanidar la columna reviews

In [ ]:
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


### 2.3.Transformacion de Datos

#### 2.3.1.Funciones para la transformacion de datos

In [ ]:
def date_post(text):

  """
  Funcion creada para extraer la fecha de la columna posted donde podemos extraer 2 formatos diferentes 1 de ellos "%B %d, %Y y el otro un formato "%B %d haciendo falta %Y
  """

  date_string =  text.lstrip('Posted ')

  if date_string.count(',')>0:
     date_str =  datetime.datetime.strptime(date_string, "%B %d, %Y.")

  else:
    #despues de un analisis  de los datos de la columna date post se concluyo que nuestra informacion se encuentra entre el año 2010-2015 y una parte de estas fechas no tienen año.
    #La mayoria son en febrero por tanto se procesara como si fuera el año 2012
    año_bisiesto  = 2012

    date_string =  date_string.replace(',', '').replace('.', '') + f', {año_bisiesto}.'
    date_str  = datetime.datetime.strptime(date_string, "%B %d, %Y.")

  return date_str



def date_edit(text):

  """
  Funcion creada para extraer la fecha de la columna last_edited donde podemos extraer 2 formatos diferentes 1 de ellos "%B %d, %Y. y el otro formato vacios
  """

  date_string = text.lstrip('Last edited ')
  if date_string.count(',')>0:

    date_str=datetime.datetime.strptime(date_string, "%B %d, %Y.")

  else:
    date_str  = np.nan
  return date_str



def extract_funny_count(text):

  """
  Funcion creada para extraer el numero de personas que calificaron como funny la review
  """

  funny_count=text.rstrip(' person found this review funny').rstrip(' people found this review funny').replace(',','')

  if len(funny_count)>0:
    funny_count =  int(funny_count)
  else:
    funny_count =  0
  return funny_count


def extract_helpful_count(text):

  """
  funcion que extrae la cantidad de votos que califican como util  la critica realizada
  """

  part1, part2 = 0, 0

  if text == 'No ratings yet':
      y=[0,0]
  else:
      vote_string = text[:text.index("p")].replace(' ', '').replace('of', ',')
      y = vote_string.split(',')
  #return [int(y[0]), int(y[1])]
  return int(y[0])



def extract_total_count(text):

  """
  funcion que extrae la cantidad de votos totales realizada a la critica
  """

  part1, part2 = 0, 0

  if text == 'No ratings yet':
      y=[0,0]
  else:
      vote_string = text[:text.index("p")].replace(' ', '').replace('of', ',')
      y = vote_string.split(',')
  return int(y[1])



def count_emojis(x):

  ''''
  Funcion creada para contar los emojis de la review
  '''

  emojis_count = []
  emojis_encontrados  = emoji.emoji_list(x)
  if emojis_encontrados == '':
      return ''
  for e in emojis_encontrados:
      emojis_count.append(e['emoji'])
  return emojis_count



def unique_emojis(emojis):

  '''
  Funcion creada para obtener los emojis unicos de la lista
  '''
  return list(set(emojis))



def emojis_to_words(text):

  '''
  Funcion creada para reemplazar los emojis por palabras
  '''

  for emoji, replacement in emoji_reemplazar.items():
      text = text.replace(emoji, replacement)
  return text





#### 2.3.2.Transformacion

In [ ]:
df_user_reviews['Posted Date']  = df_user_reviews['posted'].apply(date_post) #Extraemos la fecha de la publicacion

df_user_reviews['Date last edited'] =  df_user_reviews['last_edited'].apply(date_edit) #Extraemos la fecha de la ultima edicion

df_user_reviews['funny review votes'] = df_user_reviews['funny'].apply(extract_funny_count) #Extraemos la cantidad de votos que calificaron como divertido

df_user_reviews['Helpful review votes'] = df_user_reviews['helpful'].apply(extract_helpful_count) #Extraemos la cantidad de votos que calificaron como util

df_user_reviews['total review votes'] = df_user_reviews['helpful'].apply(extract_total_count) #Extraemos la cantidad de votos totales

df_user_reviews['Date last edited']=df_user_reviews['Date last edited'].fillna(df_user_reviews['Posted Date']) #Reemplazamos los NaN por la fecha de publicacion

df_user_reviews.drop(columns=['posted','last_edited','funny','helpful'],inplace=True) #Eliminamos las columnas que no usaremos

In [ ]:
df_user_reviews.head

<bound method NDFrame.head of                  user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
1                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

      item_id recomme

Al revisar la columna reviews podemos observar que hay varios emojis en cada review por tanto procederemos a transformarlos a texto

In [ ]:
emojis = df_user_reviews['review'].apply(lambda x: count_emojis(x))
emojis = emojis[emojis.apply(lambda x: len(x) > 0)]
emojis = emojis.apply(unique_emojis)
emojis.drop_duplicates(inplace=True)
emojis.count()


8

In [ ]:
emojis_unicos = []
for emojis_lista in emojis:
    emojis_unicos.extend(emojis_lista) # Recorremos la lista y los representamos

# Convertir la lista extendida en un conjunto para eliminar duplicados y luego en una lista nuevamente
emojis_unicos = list(set(emojis_unicos))
emojis_unicos # Nuestros emojis unicos

['©', '®', '™', '🏻']

In [ ]:
emoji_reemplazar = { # A los emojis obtenidos le agregamos los "Emojis manuales", como por ejemplo: ':)' y ':('
    # Determinamos una palabra representativa para cada emoji
    '✔': 'Approved ',
    '💯': 'Perfect ',
    '♀': 'Woman ',
    '❤': 'Love ',
    '💋': 'Kiss ',
    '💀': 'Skull ',
    '™': 'Trademark ',
    '®': 'Registered Trademark ',
    '©': 'Copyright ',
    '☺': 'Smile ',
    '♥': 'Heart ',
    '👌': 'Okay ',
    '👀': 'Eye ',
    '⌛': 'Hourglass ',
    '☹': 'Sad ',
    '☑': 'Checked ',
    '😊': 'Happiness ',
    '😄': 'Happiness ',
    '◀': 'Left ',
    ':D': 'laugh ',
    ':3': 'smile',
    'xD': 'laughloud',
    'ô•': 'oh',
    ':c': 'adictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-' : 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'adictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'
}


In [ ]:
df_user_reviews['review'] = df_user_reviews['review'].apply(emojis_to_words)

df_user_reviews['review'] = df_user_reviews['review'].str.lower() # Pasamos a minusculas las reviews

In [ ]:
df_user_reviews.head()

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,simple yet with great replayability. in my opi...,2011-11-05,2011-11-05,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,it's unique and worth a playthrough.,2011-07-15,2011-07-15,0,0,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,2011-04-21,0,0,0
1,js41637,http://steamcommunity.com/id/js41637,251610,True,i know what you think when you see this title ...,2014-06-24,2014-06-24,0,15,20
1,js41637,http://steamcommunity.com/id/js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,2013-09-08,0,0,1


#### 2.3.3.Funciones para la transformacion y preprocesamiento de texto

In [ ]:
#Llamando a las stopwords en inglés
stopwords = nltk.corpus.stopwords.words('english')
stopwords = [word for word in stopwords if word != 'not']

def delete_stopwords(x):

  """
  Funcion creada para eliminar las stopwords
  """

  return " ".join([palabra for palabra in nltk.tokenize.word_tokenize(x) if palabra not in (stopwords)])



def clean_text(x):

    '''
    Funcion creada para eliminar caracteres especiales y mantener solo letras y espacios
    '''

    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', x)
    return texto_limpio

#### 2.3.4.Preprocesamiento de texto

In [ ]:
df_user_reviews['review'] = df_user_reviews['review'].apply(delete_stopwords)

df_user_reviews['review'] = df_user_reviews['review'].apply(clean_text)

df_user_reviews.reset_index(drop=True, inplace=True)

### 3.Exportacion

Vista previa de la tabla limpia

In [ ]:
df_user_reviews

,user_id,user_url,item_id,recommend,review,Posted Date,Date last edited,funny review votes,Helpful review votes,total review votes
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,simple yet great replayability opinion zombi...,2011-11-05,2011-11-05,0,0,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,s unique worth playthrough,2011-07-15,2011-07-15,0,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,great atmosphere gunplay bit chunky times end...,2011-04-21,2011-04-21,0,0,0
3,js41637,http://steamcommunity.com/id/js41637,251610,True,know think see title barbie dreamhouse party ...,2014-06-24,2014-06-24,0,15,20
4,js41637,http://steamcommunity.com/id/js41637,227300,True,simple s actually not simple truck driving ...,2013-09-08,2013-09-08,0,0,1
...,...,...,...,...,...,...,...,...,...,...
54452,76561198312638244,http://steamcommunity.com/profiles/76561198312...,130,True,liked half life would really recommend getting...,2012-07-10,2012-07-10,0,0,0
54453,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,True,game perfect remake original half life person...,2012-07-08,2012-07-08,0,0,0
54454,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,True,much fun plaing collecting resources laughloud...,2012-07-03,2012-07-03,1,1,2
54455,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,True,laugh,2012-07-20,2012-07-20,0,0,0


In [ ]:
# Exportamos nuestro dataframe a un parquet comprimido
df_user_reviews.to_parquet('/content/drive/MyDrive/2023/Henry/MLOps Henry/Data_Clean/clean_user_reviews.parquet.gzip', compression='gzip',index=False)